In [6]:
import tensorflow as tf
import numpy as np
import os

In [10]:
shahname_file = "shahnameh.txt"

# باز کردن و خواندن محتوا
with open(shahname_file, "r", encoding="utf-8") as f:
    shahname_text = f.read()

In [12]:
import tensorflow as tf

sequence_length = 100

def split_input(input, sequence_length):
    for i in range(0, len(input), sequence_length):
        yield input[i : i + sequence_length]

features = list(split_input(shahname_text[:-1], sequence_length))
labels = list(split_input(shahname_text[1:], sequence_length))
dataset = tf.data.Dataset.from_tensor_slices((features, labels))

In [17]:
x, y = next(dataset.as_numpy_iterator())
print(x[:50].decode("utf-8", errors = "ignore")),
print(y[:50].decode("utf-8", errors = "ignore"))

|به نام خداوند جان و خرد
|کزی
به نام خداوند جان و خرد
|کزی


In [18]:
from keras import layers

tokenizer = layers.TextVectorization(
    standardize=None,
    split="character",
    output_sequence_length=sequence_length,
)
tokenizer.adapt(dataset.map(lambda text, labels: text))

In [19]:
vocabulary_size = tokenizer.vocabulary_size()
vocabulary_size

50

In [20]:
dataset = dataset.map(
    lambda features, labels: (tokenizer(features), tokenizer(labels)),
    num_parallel_calls=8,
)
training_data = dataset.shuffle(10_000).batch(64).cache()

In [ ]:
import keras

embedding_dim = 256
hidden_dim = 1024

inputs = layers.Input(shape=(sequence_length,), dtype="int", name="token_ids")
x = layers.Embedding(vocabulary_size, embedding_dim)(inputs)
x = layers.GRU(hidden_dim, return_sequences=True)(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(vocabulary_size, activation="softmax")(x)
model = keras.Model(inputs, outputs)

In [23]:
model.summary(line_length=80)

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                      ┃ Output Shape             ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ token_ids (InputLayer)            │ (None, 100)              │             0 │
├───────────────────────────────────┼──────────────────────────┼───────────────┤
│ embedding_1 (Embedding)           │ (None, 100, 256)         │        12,800 │
├───────────────────────────────────┼──────────────────────────┼───────────────┤
│ gru_1 (GRU)                       │ (None, 100, 1024)        │     3,938,304 │
├───────────────────────────────────┼──────────────────────────┼───────────────┤
│ dropout_1 (Dropout)               │ (None, 100, 1024)        │             0 │
├───────────────────────────────────┼──────────────────────────┼───────────────┤
│ dense_1 (Dense)                   │ (None, 100, 50)          │        51,250 │
└───────────────────────────────────┴──────────────────────────┴───────────────┘

 Total params: 4,002,354 (15.27 MB)

 Trainable params: 4,002,354 (15.27 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["sparse_categorical_accuracy"],
)
model.fit(training_data, epochs=20) 

Epoch 1/20
415/415 ━━━━━━━━━━━━━━━━━━━━ 386s 924ms/step - loss: 1.9876 - sparse_categorical_accuracy: 0.4265
Epoch 2/20
415/415 ━━━━━━━━━━━━━━━━━━━━ 356s 857ms/step - loss: 1.4365 - sparse_categorical_accuracy: 0.5665
Epoch 3/20
415/415 ━━━━━━━━━━━━━━━━━━━━ 361s 869ms/step - loss: 1.3188 - sparse_categorical_accuracy: 0.5983
Epoch 4/20
415/415 ━━━━━━━━━━━━━━━━━━━━ 370s 891ms/step - loss: 1.2558 - sparse_categorical_accuracy: 0.6159
Epoch 5/20
415/415 ━━━━━━━━━━━━━━━━━━━━ 362s 873ms/step - loss: 1.2085 - sparse_categorical_accuracy: 0.6293
Epoch 6/20
415/415 ━━━━━━━━━━━━━━━━━━━━ 361s 869ms/step - loss: 1.1672 - sparse_categorical_accuracy: 0.6412
Epoch 7/20
415/415 ━━━━━━━━━━━━━━━━━━━━ 358s 863ms/step - loss: 1.1300 - sparse_categorical_accuracy: 0.6518
Epoch 8/20
415/415 ━━━━━━━━━━━━━━━━━━━━ 368s 886ms/step - loss: 1.0960 - sparse_categorical_accuracy: 0.6616
Epoch 9/20
415/415 ━━━━━━━━━━━━━━━━━━━━ 359s 865ms/step - loss: 1.0636 - sparse_categorical_accuracy: 0.6710
Epoch 10/20
415/415

In [25]:
inputs = keras.Input(shape=(1,), dtype="int", name="token_ids")
input_state = keras.Input(shape=(hidden_dim,), name="state")

x = layers.Embedding(vocabulary_size, embedding_dim)(inputs)
x, output_state = layers.GRU(hidden_dim, return_state=True)(
    x, initial_state=input_state
)
outputs = layers.Dense(vocabulary_size, activation="softmax")(x)
generation_model = keras.Model(
    inputs=(inputs, input_state),
    outputs=(outputs, output_state),
)
generation_model.set_weights(model.get_weights())

In [26]:
tokens = tokenizer.get_vocabulary()
token_ids = range(vocabulary_size)
char_to_id = dict(zip(tokens, token_ids))
id_to_char = dict(zip(token_ids, tokens))

prompt = """
به نام خدا
"""

In [27]:
input_ids = [char_to_id[c] for c in prompt]
state = keras.ops.zeros(shape=(1, hidden_dim))
for token_id in input_ids:
    inputs = keras.ops.expand_dims([token_id], axis=0)
    predictions, state = generation_model.predict((inputs, state), verbose=0)

In [28]:
import numpy as np

generated_ids = []
max_length = 250
for i in range(max_length):
    next_char = int(np.argmax(predictions, axis=-1)[0])
    generated_ids.append(next_char)
    inputs = keras.ops.expand_dims([next_char], axis=0)
    predictions, state = generation_model.predict((inputs, state), verbose=0)

In [29]:
output = "".join([id_to_char[token_id] for token_id in generated_ids])
print(prompt + output)


به نام خدا
|نه کهتر بود مردم پارسا
|به نیکی برافگند پویان به راه
|به درگاه ایوان و بالای اوی
|بران رای بر پای بر با سپاه
|بیامد بر سام و بر سر گرفت
|به روم اندرون پیل و چنگال شیر
|بر پادشا را ببستم بروی
|برآمد ز هر دو سپه بوی و رنگ
|ز دریا برآمد یکی تیره گرد
|ب
